In [ ]:
%load_ext lab_black
import pandas as pd
import numpy as np
import os
import statsmodels.api as sm
import subprocess
from tqdm import tqdm
import itertools
import matplotlib.pyplot as plt

In [ ]:
group = "white"

trait, covar = "log_BMI", "SEX"
# trait, covar = "avMSE", "glasses"

In [ ]:
df = pd.read_csv(
    f"out/estimate-quantify/{trait}.{group}.pred.tsv",
    sep="\t",
    index_col=0,
)
if len(df[covar].unique()) > 5:
    df["q"] = pd.qcut(df[covar], q=5).cat.codes
else:
    df["q"] = df[covar].values

In [ ]:
df_stats = {
    "r2": [],
    "std(resid)": [],
    "std(resid)2": [],
    "std(pred)": [],
    "std(y)": [],
    "slope": [],
}

for i, (q, dfq) in enumerate(df.groupby("q")):
    x, y = dfq["pred_mean"], dfq["PHENO"]
    model = sm.OLS(endog=y, exog=sm.add_constant(x)).fit()
    df_stats["r2"].append(model.rsquared)
    df_stats["std(resid)"].append(np.std(model.resid))
    df_stats["std(resid)2"].append(np.std(y - x))
    df_stats["std(pred)"].append(np.std(x))
    df_stats["std(y)"].append(np.std(y))

    model = sm.WLS(
        endog=y, exog=sm.add_constant(x), weights=(1 / dfq["pred_std"]) ** 2
    ).fit()
    df_stats["slope"].append(model.params[1])
df_stats = pd.DataFrame(df_stats)